In [8]:
"""
E-commerce Recommender System - Streamlit Demo
File: streamlit_app.py
Run with: streamlit run streamlit_app.py
"""

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from recommender_engine import RecommenderEngine, LLMExplainer
import sys

# Page configuration
st.set_page_config(
    page_title="E-commerce Recommender Demo",
    page_icon="🛍️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS
st.markdown("""
    <style>
    .main-header {
        font-size: 3rem;
        font-weight: bold;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 1rem;
    }
    .sub-header {
        text-align: center;
        color: #666;
        margin-bottom: 2rem;
    }
    .product-card {
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
        padding: 1.5rem;
        border-radius: 15px;
        margin: 1rem 0;
        border-left: 5px solid #1f77b4;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .metric-card {
        background-color: #e3f2fd;
        padding: 1.5rem;
        border-radius: 10px;
        text-align: center;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    .success-box {
        background-color: #d4edda;
        border: 1px solid #c3e6cb;
        border-radius: 8px;
        padding: 1rem;
        margin: 1rem 0;
    }
    </style>
""", unsafe_allow_html=True)

# Initialize session state
if 'engine' not in st.session_state:
    st.session_state.engine = None
if 'explainer' not in st.session_state:
    st.session_state.explainer = None
if 'recommendations_generated' not in st.session_state:
    st.session_state.recommendations_generated = False

# Load recommendation engine
@st.cache_resource
def load_engine():
    """Load recommendation engine (cached)"""
    try:
        engine = RecommenderEngine('ecommerce_recommender.db')
        explainer = LLMExplainer()
        return engine, explainer
    except Exception as e:
        return None, str(e)

# Header
st.markdown('<h1 class="main-header">🛍️ E-commerce Product Recommender</h1>', unsafe_allow_html=True)
st.markdown('<p class="sub-header">AI-Powered Hybrid Recommendation System with Explainable AI</p>', unsafe_allow_html=True)

# Load engine
engine, explainer_or_error = load_engine()

if engine is None:
    st.error(f"""
    ❌ **Database not found!**
    
    Error: {explainer_or_error}
    
    **Please follow these steps:**
    1. Open your Jupyter notebook
    2. Run ALL cells (Cell 1 through Cell 17)
    3. Wait for "Database initialized successfully" message
    4. Verify `ecommerce_recommender.db` file exists
    5. Restart this Streamlit app
    """)
    st.stop()

explainer = explainer_or_error
st.session_state.engine = engine
st.session_state.explainer = explainer

# Get statistics
stats = engine.get_statistics()

# Sidebar
st.sidebar.title("⚙️ Configuration")
st.sidebar.markdown("---")

user_id = st.sidebar.number_input(
    "Select User ID",
    min_value=1,
    max_value=stats['total_users'],
    value=1,
    help="Choose a user to generate personalized recommendations"
)

n_recommendations = st.sidebar.slider(
    "Number of Recommendations",
    min_value=3,
    max_value=10,
    value=5,
    help="How many products to recommend"
)

recommendation_method = st.sidebar.selectbox(
    "Recommendation Algorithm",
    ["Hybrid (Collaborative + Content)", "Collaborative Filtering", "Content-Based Filtering"],
    help="Choose the recommendation strategy"
)

show_explanations = st.sidebar.checkbox(
    "Show AI Explanations",
    value=True,
    help="Display why each product is recommended"
)

st.sidebar.markdown("---")
st.sidebar.info("💡 **Tip:** Try different users and methods to see how recommendations change!")

# Statistics in sidebar
st.sidebar.markdown("### 📊 System Stats")
st.sidebar.metric("Total Products", stats['total_products'])
st.sidebar.metric("Active Users", stats['total_users'])
st.sidebar.metric("Total Purchases", stats['total_purchases'])

# Main tabs
tab1, tab2, tab3, tab4 = st.tabs(["🎯 Generate Recommendations", "👤 User Profile", "📊 System Dashboard", "ℹ️ About"])

# ============================================================================
# TAB 1: RECOMMENDATIONS
# ============================================================================
with tab1:
    st.header(f"Personalized Recommendations for User {user_id}")
    
    col1, col2 = st.columns([3, 1])
    
    with col1:
        st.markdown(f"""
        **Current Settings:**
        - Algorithm: `{recommendation_method}`
        - Number of recommendations: `{n_recommendations}`
        - AI Explanations: `{'Enabled' if show_explanations else 'Disabled'}`
        """)
    
    with col2:
        generate_button = st.button(
            "🎲 Generate Recommendations",
            type="primary",
            use_container_width=True
        )
    
    if generate_button:
        with st.spinner("🔮 Analyzing user preferences and generating recommendations..."):
            try:
                # Get recommendations based on selected method
                if recommendation_method == "Collaborative Filtering":
                    product_ids = engine.collaborative_filtering(user_id, n_recommendations)
                elif recommendation_method == "Content-Based Filtering":
                    product_ids = engine.content_based_filtering(user_id, n_recommendations)
                else:  # Hybrid
                    product_ids = engine.hybrid_recommendations(user_id, n_recommendations)
                
                products = engine.get_product_details(product_ids)
                
                if len(products) == 0:
                    st.warning(f"⚠️ Could not generate recommendations for User {user_id}. Try a different user or method.")
                else:
                    st.session_state.recommendations_generated = True
                    
                    # Success message
                    st.markdown(f'<div class="success-box">✅ Successfully generated <b>{len(products)}</b> personalized recommendations!</div>', unsafe_allow_html=True)
                    
                    # Display recommendations
                    st.markdown("---")
                    
                    for idx, product in enumerate(products, 1):
                        st.markdown(f'<div class="product-card">', unsafe_allow_html=True)
                        
                        col_left, col_right = st.columns([3, 1])
                        
                        with col_left:
                            st.markdown(f"### {idx}. {product['name']}")
                            st.markdown(f"**Category:** {product['category']}")
                            st.markdown(f"**Description:** {product['description']}")
                            
                            if show_explanations:
                                user_history = engine.get_user_history(user_id)
                                explanation = explainer.generate_explanation(
                                    engine, product, user_history
                                )
                                st.info(f"🤖 **Why we recommend this:**\n\n{explanation}")
                        
                        with col_right:
                            st.metric("Price", f"${product['price']:.2f}")
                            st.metric("Rating", f"⭐ {product['rating']}/5")
                            
                            # Add to cart button (demo)
                            if st.button(f"🛒 Add to Cart", key=f"cart_{idx}"):
                                st.success("Added to cart! (Demo)")
                        
                        st.markdown('</div>', unsafe_allow_html=True)
            
            except Exception as e:
                st.error(f"❌ Error generating recommendations: {e}")

# ============================================================================
# TAB 2: USER PROFILE
# ============================================================================
with tab2:
    st.header(f"User {user_id} Activity Profile")
    
    user_history = engine.get_user_history(user_id)
    
    if len(user_history) == 0:
        st.warning(f"⚠️ User {user_id} has no activity history.")
    else:
        # Activity metrics
        col1, col2, col3 = st.columns(3)
        
        views = len(user_history[user_history['interaction_type'] == 'view'])
        purchases = len(user_history[user_history['interaction_type'] == 'purchase'])
        total = len(user_history)
        
        with col1:
            st.markdown('<div class="metric-card">', unsafe_allow_html=True)
            st.metric("Total Interactions", total)
            st.markdown('</div>', unsafe_allow_html=True)
        
        with col2:
            st.markdown('<div class="metric-card">', unsafe_allow_html=True)
            st.metric("Product Views", views)
            st.markdown('</div>', unsafe_allow_html=True)
        
        with col3:
            st.markdown('<div class="metric-card">', unsafe_allow_html=True)
            st.metric("Purchases Made", purchases)
            conversion_rate = (purchases / total * 100) if total > 0 else 0
            st.caption(f"Conversion Rate: {conversion_rate:.1f}%")
            st.markdown('</div>', unsafe_allow_html=True)
        
        st.markdown("---")
        
        # Purchase history
        if purchases > 0:
            st.subheader("🛒 Purchase History")
            
            purchased_ids = user_history[user_history['interaction_type'] == 'purchase']['product_id'].values
            purchased_products = engine.get_product_details(purchased_ids)
            
            for product in purchased_products:
                col_a, col_b, col_c = st.columns([2, 1, 1])
                with col_a:
                    st.write(f"**{product['name']}**")
                with col_b:
                    st.write(f"${product['price']:.2f}")
                with col_c:
                    st.write(f"⭐ {product['rating']}/5")
        
        st.markdown("---")
        
        # Recent activity table
        st.subheader("📝 Recent Activity")
        recent_activity = user_history.sort_values('timestamp', ascending=False).head(10)
        st.dataframe(
            recent_activity[['product_id', 'interaction_type', 'timestamp', 'rating']],
            use_container_width=True
        )

# ============================================================================
# TAB 3: DASHBOARD
# ============================================================================
with tab3:
    st.header("System Performance Dashboard")
    
    # Top metrics
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.metric("Total Products", stats['total_products'])
        st.markdown('</div>', unsafe_allow_html=True)
    
    with col2:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.metric("Active Users", stats['total_users'])
        st.markdown('</div>', unsafe_allow_html=True)
    
    with col3:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.metric("Total Purchases", stats['total_purchases'])
        st.markdown('</div>', unsafe_allow_html=True)
    
    with col4:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.metric("Avg Rating", f"{stats['avg_rating']:.2f}/5")
        st.markdown('</div>', unsafe_allow_html=True)
    
    st.markdown("---")
    
    # Charts
    col_left, col_right = st.columns(2)
    
    with col_left:
        st.subheader("📦 Product Distribution by Category")
        category_counts = engine.products_df['category'].value_counts()
        fig1 = px.pie(
            values=category_counts.values,
            names=category_counts.index,
            title="Product Categories"
        )
        st.plotly_chart(fig1, use_container_width=True)
    
    with col_right:
        st.subheader("💰 Price Distribution")
        fig2 = px.histogram(
            engine.products_df,
            x='price',
            nbins=20,
            title="Product Price Range"
        )
        st.plotly_chart(fig2, use_container_width=True)
    
    st.markdown("---")
    
    # Product catalog
    st.subheader("📋 Complete Product Catalog")
    st.dataframe(
        engine.products_df[['name', 'category', 'price', 'rating', 'description']],
        use_container_width=True,
        height=400
    )

# ============================================================================
# TAB 4: ABOUT
# ============================================================================
with tab4:
    st.header("About This System")
    
    st.markdown("""
    ## 🎯 E-commerce Product Recommender System
    
    This is a **hybrid recommendation system** that combines multiple AI techniques to provide personalized product suggestions.
    
    ### 🧠 Algorithms Used
    
    #### 1. **Collaborative Filtering**
    - Analyzes user behavior patterns
    - Finds similar users based on purchase history
    - Recommends products liked by similar users
    - **Strength:** Discovers unexpected connections
    
    #### 2. **Content-Based Filtering**
    - Analyzes product attributes (category, description)
    - Uses TF-IDF vectorization for text analysis
    - Calculates product similarity using cosine similarity
    - **Strength:** Provides consistent, explainable recommendations
    
    #### 3. **Hybrid Approach** (Recommended)
    - Combines both methods (60% collaborative, 40% content-based)
    - Balances discovery and relevance
    - **Strength:** Best overall performance
    
    ### 📊 Evaluation Metrics
    
    - **Coverage:** Percentage of catalog recommended
    - **Diversity:** Variety of categories in recommendations
    - **Quality:** Average rating of recommended products
    
    ### 🛠️ Technologies
    
    - **Backend:** Python, Pandas, NumPy, Scikit-learn
    - **Database:** SQLite
    - **UI:** Streamlit
    - **ML:** TF-IDF, Cosine Similarity
    - **Visualization:** Plotly
    
    ### 📝 How to Use
    
    1. **Select a user** from the sidebar (1-50)
    2. **Choose recommendation method** (Hybrid recommended)
    3. **Click "Generate Recommendations"**
    4. **Explore** user profiles and system statistics
    
    ### 🎓 Created By
    
    This system was built as a demonstration of modern recommendation algorithms used by e-commerce platforms like Amazon, Netflix, and Spotify.
    
    ---
    
    **Version:** 1.0  
    **Last Updated:** 2025
    """)

# Footer
st.markdown("---")
st.markdown("""
<div style='text-align: center; color: gray; padding: 2rem;'>
    <p style='font-size: 0.9rem;'>
        Built with ❤️ using Streamlit | E-commerce Recommender System Demo
    </p>
    <p style='font-size: 0.8rem;'>
        Hybrid Recommendation Engine + Explainable AI
    </p>
</div>
""", unsafe_allow_html=True)

2025-10-17 00:13:40.490 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.495 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.497 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2025-10-17 00:13:40.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 00:13:40.541 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()